# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv from part 1
file = os.path.join('..', 'output_data', 'cities.csv')

# Read and display the dataframe
cities_df = pd.read_csv(file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,quesnel,52.9999,-122.5029,28.40,100,90,2.06,CA,1612573797
1,kalat,29.0225,66.5916,34.57,51,0,1.86,PK,1612573798
2,kapaa,22.0752,-159.3190,71.60,56,1,5.75,US,1612573634
3,oruro,-17.9833,-67.1500,52.79,63,28,14.45,BO,1612573718
4,longyearbyen,78.2186,15.6401,28.40,69,75,16.11,SJ,1612573798


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Set locations with lat and long from cities DataFrame
locations = cities_df[['Lat', 'Lng']]

#Set weight for gmap layer (humidity)
humidity = cities_df['Humidity']

# Plot gmaps with given info
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,
                                 max_intensity=100, point_radius=2
                                 )

# Add heat layer to gmaps
fig.add_layer(heat_layer)

#display gmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Find cities with max temps between 70 and 80
ideal_con = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70), :]

# Add to existing ideal DataFrame and find wind speeds below 10 mph
ideal_con = ideal_con.loc[ideal_con['Wind Speed'] < 10]

# Keep adding to the ideal DataFrame and find locations with zero cloudiness
ideal_con = ideal_con.loc[ideal_con['Cloudiness'] == 0]

# Display ideal DataFrame
ideal_con

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,zabid,14.1951,43.3152,70.27,80,0,5.75,YE,1612573800
57,san lorenzo,-25.3333,-57.5333,71.60,73,0,3.44,PY,1612573807
59,saint-philippe,-21.3585,55.7679,78.80,65,0,6.91,RE,1612573807
164,vao,-22.6667,167.4833,78.60,83,0,9.37,NC,1612573822
216,marawi,7.9986,124.2928,79.00,79,0,4.00,PH,1612573829
300,santa helena de goias,-17.8136,-50.5969,73.11,90,0,5.77,BR,1612573840
306,tabou,4.4230,-7.3528,78.78,91,0,7.07,CI,1612573841
312,harper,4.3750,-7.7169,79.14,89,0,8.39,LR,1612573755
324,port blair,11.6667,92.7500,79.11,74,0,8.19,IN,1612573844
345,coihaique,-45.5752,-72.0662,73.40,46,0,2.30,CL,1612573846


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store ideal conditions df into new variable and add new column
hotel_df = ideal_con
hotel_df['Hotel Name'] = ''

# Display new dataframe with added column
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,zabid,14.1951,43.3152,70.27,80,0,5.75,YE,1612573800,
57,san lorenzo,-25.3333,-57.5333,71.60,73,0,3.44,PY,1612573807,
59,saint-philippe,-21.3585,55.7679,78.80,65,0,6.91,RE,1612573807,
164,vao,-22.6667,167.4833,78.60,83,0,9.37,NC,1612573822,
216,marawi,7.9986,124.2928,79.00,79,0,4.00,PH,1612573829,
300,santa helena de goias,-17.8136,-50.5969,73.11,90,0,5.77,BR,1612573840,
306,tabou,4.4230,-7.3528,78.78,91,0,7.07,CI,1612573841,
312,harper,4.3750,-7.7169,79.14,89,0,8.39,LR,1612573755,
324,port blair,11.6667,92.7500,79.11,74,0,8.19,IN,1612573844,
345,coihaique,-45.5752,-72.0662,73.40,46,0,2.30,CL,1612573846,


In [ ]:
# Google maps example query search for reference when making params
# https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

In [13]:
# Create hotel names to add to and set query params
hotel_names = []
params = {"radius": 5000, 
          "type": "hotel",
          "keyword": "hotel",          
          "key": g_key}
print('Start of Hotel search..')
print('---------------------------------')
# Loop through lat and long with iterrows to find nearest hotel
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    
    #set base url and response
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(url, params=params).json()
    
    #extract the results from API response
    results = response['results']
    
    try:
        hotel_names.append(results[0]['name'])
        print(f'The closest hotel at index {index} is: {results[0]["name"]}')
    except(IndexError):
        hotel_names.append('')
        print(f'Hotel was not found.. at index {index}: {lat}, {long}')
print('---------------------------------')
print('End of search..')

Start of Hotel search..
---------------------------------
Hotel was not found.. at index 21: 14.1951, 43.3152
The closest hotel at index 57 is: Factoria Hotel
The closest hotel at index 59 is: Les Embruns Du Baril
The closest hotel at index 164 is: OURE TERA beach resort
The closest hotel at index 216 is: Marawi Resort Hotel
The closest hotel at index 300 is: Hotel Pacheco Santa Helena GO
The closest hotel at index 306 is: hotêl doufoulougou
The closest hotel at index 312 is: Grands' Guesthouse
The closest hotel at index 324 is: Welcomhotel Bay Island Port Blair
The closest hotel at index 345 is: Patagonia House
The closest hotel at index 394 is: Playa 55 beach escape
The closest hotel at index 456 is: Hotel Gualok
The closest hotel at index 499 is: Hotel - Miro Park
The closest hotel at index 520 is: Design Suites Bariloche
The closest hotel at index 532 is: Hotel Le Mont Sincery De Dabola.
---------------------------------
End of search..


In [14]:
# Add results from API response to hotel DataFrame..
hotel_df['Hotel Name'] = hotel_names
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,zabid,14.1951,43.3152,70.27,80,0,5.75,YE,1612573800,
57,san lorenzo,-25.3333,-57.5333,71.60,73,0,3.44,PY,1612573807,Factoria Hotel
59,saint-philippe,-21.3585,55.7679,78.80,65,0,6.91,RE,1612573807,Les Embruns Du Baril
164,vao,-22.6667,167.4833,78.60,83,0,9.37,NC,1612573822,OURE TERA beach resort
216,marawi,7.9986,124.2928,79.00,79,0,4.00,PH,1612573829,Marawi Resort Hotel
300,santa helena de goias,-17.8136,-50.5969,73.11,90,0,5.77,BR,1612573840,Hotel Pacheco Santa Helena GO
306,tabou,4.4230,-7.3528,78.78,91,0,7.07,CI,1612573841,hotêl doufoulougou
312,harper,4.3750,-7.7169,79.14,89,0,8.39,LR,1612573755,Grands' Guesthouse
324,port blair,11.6667,92.7500,79.11,74,0,8.19,IN,1612573844,Welcomhotel Bay Island Port Blair
345,coihaique,-45.5752,-72.0662,73.40,46,0,2.30,CL,1612573846,Patagonia House


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))